# ייבוא ספריות שימושיות

In [30]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set style for visualizations - using seaborn directly
sns.set_style("darkgrid")
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")


✅ Libraries imported successfully!
Pandas version: 1.4.4


# קריאת הנתונים ומבט ראשוני

In [44]:
url = "https://raw.githubusercontent.com/Roybin12/data-ISOC-IL/refs/heads/main/data.csv"
df = pd.read_csv(url)
df.head()

,id,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q10,Q11,Q11.1,Q15,Q15.1,Q15.2,Q15.3,Q15.4,Q15.5,Q15.6,Q15.7,Q15.8,Q15.9,Q15.10,Q15.11,Q16,Q16.1,Q16.2,Q16.3,Q16.4,Q16.5,Q16.6,Q16.7,Q16.8,Q16.9,Q17,Q17.1,Q17.2,Q17.3,Q17.4,Q17.5,Q17.6,Q17.7,Q18,Q20,Q21,device_type,מקור,ISAIncome
0,259,אישה,18,ערבי - מוסלמי,הדרום,NaN,NaN,1,לא יודע/ת,לא יודע/ת,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,לא יודע ת,NaN,NaN,NaN,יוטיוב - YouTube - מנוי בתשלום,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,אף אחד מהם,על תיכונית,לא יודע/ת,הרבה מתחת לממוצע,"mobile, iOS",0,עד 3000
1,319,גבר,18,יהודי,NaN,(04) צפון,חילוני/ת,1,לא יודע/ת,לא יודע/ת,NaN,NaN,NaN,יס - Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,אף אחד מהם,ספוטיפיי – Spotify,NaN,YouTube Music YouTube Premium,NaN,NaN,NaN,NaN,NaN,על תיכונית,לא יודע/ת,מסרב/ת להשיב,desktop,0,לא יודע|ת
2,403,גבר,18,יהודי,NaN,(03) מרכז,חילוני/ת,1,לא יודע/ת,לא יודע/ת,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,לא יודע ת,NaN,נטפליקס – Netflix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ספוטיפיי – Spotify,NaN,NaN,NaN,NaN,NaN,NaN,NaN,תיכונית / ישיבה קטנה,עד 3000,NaN,"mobile, Android",0,עד 3000
3,568,אישה,18,יהודי,NaN,(03) מרכז,חילוני/ת,1,סיב אופטי,סלקום,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,אחר,NaN,NaN,נטפליקס – Netflix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ספוטיפיי – Spotify,NaN,NaN,NaN,NaN,NaN,NaN,NaN,תיכונית / ישיבה קטנה,עד 3000,NaN,"mobile, iOS",0,עד 3000
4,641,אישה,18,יהודי,NaN,(03) מרכז,מסורתי/ת,1,קו טלפון (ADSL),הוט נט,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FreeTV,NaN,NaN,NaN,NaN,נטפליקס – Netflix,דיסני פלוס,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ספוטיפיי – Spotify,אפל מיוזיק - Apple music,NaN,NaN,NaN,NaN,NaN,NaN,תיכונית / ישיבה קטנה,4001-5000,NaN,"mobile, Android",0,4001-5000


# איחוד עמודות כפולות בנתוני הסקר
#### במהלך איסוף הנתונים נוצרו עמודות כפולות ומפוצלות שמקשות על הניתוח. שלב זה מטפל בניקוי והסדרת הנתונים לצורך ניתוח יעיל.
## פעולות שיבוצעו:

#### איחוד עמודות 5-6 - שתי עמודות של אותה שאלה שפוצלו בטעות
#### הסרת כפילויות - מחיקת עמודה 11 הכפולה והריקה
#### איחוד שאלות רב-ברירה - צירוף עמודות 15 (12 אופציות), 16 (9 אופציות), 17 (8 אופציות)

In [45]:
# איחוד עמודות כפולות בנתוני הסקר - גרסה מתוקנת

import pandas as pd

# 1. איחוד עמודות 5-6 (אותה שאלה)
col5_name = df.columns[4]  # עמודה 5
col6_name = df.columns[5]  # עמודה 6

# המרה למחרוזות ואיחוד
df[col5_name] = df[col5_name].astype(str).replace('nan', '').replace('None', '')
df[col6_name] = df[col6_name].astype(str).replace('nan', '').replace('None', '')

df['merged_Q5_Q6'] = (df[col5_name] + ' ' + df[col6_name]).str.strip()
df['merged_Q5_Q6'] = df['merged_Q5_Q6'].replace('', None)

# מחיקת העמודות המקוריות
df = df.drop(columns=[col5_name, col6_name])

# 2. הסרת עמודות כפולות ספציפיות
# הסרת עמודה 11.1
cols_to_remove = []
for col in df.columns:
    if '11.1' in str(col) or 'Q11.1' in str(col):
        cols_to_remove.append(col)

if cols_to_remove:
    df = df.drop(columns=cols_to_remove)
   

# הסרת עמודה 16.9
cols_to_remove_16_9 = []
for col in df.columns:
    if '16.9' in str(col) or 'Q16.9' in str(col):
        cols_to_remove_16_9.append(col)

if cols_to_remove_16_9:
    df = df.drop(columns=cols_to_remove_16_9)
   

# הסרת עמודות 11 כפולות נוספות
col11_duplicates = []
col11_names = [col for col in df.columns if 'Q11' in str(col) and '11.1' not in str(col)]
if len(col11_names) > 1:
    # שמירת הראשונה, מחיקת השאר
    col11_duplicates = col11_names[1:]
    df = df.drop(columns=col11_duplicates)
    

# 3. איחוד עמודה 15 (12 אופציות)
q15_cols = [col for col in df.columns if 'Q15' in str(col)]

if len(q15_cols) >= 12:
    def combine_q15(row):
        answers = []
        for col in q15_cols[:12]:  # רק 12 הראשונות
            val = str(row[col]) if pd.notna(row[col]) else ''
            val = val.replace('nan', '').replace('None', '').strip()
            if val and val != '':
                answers.append(val)
        return '; '.join(answers) if answers else None
    
    df['merged_Q15'] = df.apply(combine_q15, axis=1)
    df = df.drop(columns=q15_cols[:12])
    

# 4. איחוד עמודה 16 (9 אופציות) - ללא 16.9
q16_cols = [col for col in df.columns if 'Q16' in str(col) and '16.9' not in str(col)]

if len(q16_cols) >= 9:
    def combine_q16(row):
        answers = []
        for col in q16_cols[:9]:  # רק 9 הראשונות
            val = str(row[col]) if pd.notna(row[col]) else ''
            val = val.replace('nan', '').replace('None', '').strip()
            if val and val != '':
                answers.append(val)
        return '; '.join(answers) if answers else None
    
    df['merged_Q16'] = df.apply(combine_q16, axis=1)
    df = df.drop(columns=q16_cols[:9])
    


# 5. איחוד עמודה 17 (8 אופציות)
q17_cols = [col for col in df.columns if 'Q17' in str(col)]

if len(q17_cols) >= 8:
    def combine_q17(row):
        answers = []
        for col in q17_cols[:8]:  # רק 8 הראשונות
            val = str(row[col]) if pd.notna(row[col]) else ''
            val = val.replace('nan', '').replace('None', '').strip()
            if val and val != '':
                answers.append(val)
        return '; '.join(answers) if answers else None
    
    df['merged_Q17'] = df.apply(combine_q17, axis=1)
    df = df.drop(columns=q17_cols[:8])
    

# 6. בדיקה סופית לעמודות שנשארו
remaining_problematic = []
for col in df.columns:
    if '11.1' in str(col) or '16.9' in str(col):
        remaining_problematic.append(col)

if remaining_problematic:
    df = df.drop(columns=remaining_problematic)
    


#הסרת עמודת Q8
df.drop(columns=["Q8"], inplace=True)
 


# שינוי שמות עמודות לעברית
## רקע השלב:
לאחר שסיימנו לאחד את העמודות הכפולות, כעת נמיר את שמות העמודות מקודים באנגלית לשמות ברורים בעברית. שלב זה חיוני להבנת הנתונים ולביצוע ניתוח יעיל.
## תהליך השינוי:
הקוד יעבור על כל עמודה ויחליף את שמה הנוכחי בשם עברי מתאים. בסוף התהליך נקבל דאטה עם שמות עמודות ברורים ומובנים שיקלו על המשך העבודה.

In [46]:
# שינוי שמות עמודות לעברית

# מילון התרגום משמות קיימים לשמות חדשים בעברית
column_mapping = {
    'id': 'ת״ז',
    'Q2': 'מגדר',
    'Q3': 'גיל',
    'Q4': 'לאום',
    'merged_Q5_Q6': 'איזור מגורים',
    'Q7': 'הגדרה דתית',
    'Q10': 'סוג חיבור האינטרנט',
    'Q11': 'ספק האינטרנט',
    'merged_Q15': 'ספק טלויזיה ישראלי',
    'merged_Q16': 'ספק טלויזיה בינלאומי',
    'merged_Q17': 'צריכת שירותי מוזיקה',
    'Q18': 'השכלה',
    'Q20': 'הכנסה למשק בית',
    'Q21': 'הכנסה מעל או מתחת לממוצע',
    'device_type': 'סוג מכשיר',
    'מקור': 'מקור',
    'ISAIncome': 'הכנסה'
}

# ביצוע שינוי השמות
df_renamed = df.rename(columns=column_mapping)

# העתקת הדאטה המעודכנת
df = df_renamed.copy()

df

,ת״ז,מגדר,גיל,לאום,הגדרה דתית,סוג חיבור האינטרנט,ספק האינטרנט,השכלה,הכנסה למשק בית,הכנסה מעל או מתחת לממוצע,סוג מכשיר,מקור,הכנסה,איזור מגורים,ספק טלויזיה ישראלי,ספק טלויזיה בינלאומי,צריכת שירותי מוזיקה
0,259,אישה,18,ערבי - מוסלמי,NaN,לא יודע/ת,לא יודע/ת,על תיכונית,לא יודע/ת,הרבה מתחת לממוצע,"mobile, iOS",0,עד 3000,הדרום,לא יודע ת,יוטיוב - YouTube - מנוי בתשלום,אף אחד מהם
1,319,גבר,18,יהודי,חילוני/ת,לא יודע/ת,לא יודע/ת,על תיכונית,לא יודע/ת,מסרב/ת להשיב,desktop,0,לא יודע|ת,(04) צפון,יס - Yes,None,ספוטיפיי – Spotify; YouTube Music YouTube Pr...
2,403,גבר,18,יהודי,חילוני/ת,לא יודע/ת,לא יודע/ת,תיכונית / ישיבה קטנה,עד 3000,NaN,"mobile, Android",0,עד 3000,(03) מרכז,לא יודע ת,נטפליקס – Netflix,ספוטיפיי – Spotify
3,568,אישה,18,יהודי,חילוני/ת,סיב אופטי,סלקום,תיכונית / ישיבה קטנה,עד 3000,NaN,"mobile, iOS",0,עד 3000,(03) מרכז,אחר,נטפליקס – Netflix,ספוטיפיי – Spotify
4,641,אישה,18,יהודי,מסורתי/ת,קו טלפון (ADSL),הוט נט,תיכונית / ישיבה קטנה,4001-5000,NaN,"mobile, Android",0,4001-5000,(03) מרכז,FreeTV,נטפליקס – Netflix; דיסני פלוס,ספוטיפיי – Spotify; אפל מיוזיק - Apple music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,1014,גבר,72,יהודי,מסורתי/ת,סיב אופטי,סלקום,אקדמאית תואר ראשון / ישיבה גבוהה / כולל,18001-22000,NaN,"mobile, iOS",0,18001-22000,(04) צפון,סלקום TV,None,ספוטיפיי – Spotify
1008,402,גבר,74,יהודי,חילוני/ת,סיב אופטי,בזק בינלאומי,על תיכונית,12001-13000,NaN,"mobile, Android",0,12001-13000,(04) צפון,Sting TV,None,אחר
1009,43,אישה,76,יהודי,דתי/ה,אחר,אקספון 018 (wecom),על תיכונית,3001-4000,NaN,desktop,0,3001-4000,(08) דרום,אף אחד מהם,יוטיוב - YouTube - מנוי בתשלום,ספוטיפיי – Spotify; YouTube Music YouTube Pr...
1010,533,גבר,77,יהודי,מסורתי/ת,סיב אופטי,בזק בינלאומי,אקדמאית תואר ראשון / ישיבה גבוהה / כולל,32001-36000,NaN,desktop,0,32001-36000,(09) שרון,יס - Yes,None,אף אחד מהם


# ניתוח ערכים חסרים בנתוני הסקר
לאחר שסידרנו את מבנה הנתונים ושינינו את שמות העמודות לעברית, הגיע הזמן לבחון את איכות הנתונים עצמם. אחת הבעיות הנפוצות ביותר בנתוני סקרים היא קיומם של ערכים חסרים - משיבים שלא ענו על שאלות מסוימות או שהשאירו תשובות ריקות.
הניתוח הבא יעזור לנו להבין את דפוסי הערכים החסרים בכל עמודה ולקבל החלטות מושכלות לגבי אופן הטיפול בהם. זה שלב קריטי שיקבע את האסטרטגיה שלנו להמשך הניתוח  .

In [49]:
# ניתוח מקיף של ערכים חסרים

print("🔍 ניתוח ערכים חסרים בנתוני הסקר")
print("=" * 60)

# 1. סקירה כללית של הנתונים
total_rows = len(df)
total_cols = len(df.columns)
print(f"📊 מימדי הדאטה: {total_rows:,} שורות, {total_cols} עמודות")
print("-" * 60)

# 2. חישוב ערכים חסרים לכל עמודה
missing_data = pd.DataFrame({
    'עמודה': df.columns,
    'ערכים_חסרים': df.isnull().sum(),
    'אחוז_חסר': (df.isnull().sum() / total_rows * 100).round(2),
    'ערכים_קיימים': df.count(),
    'אחוז_קיים': (df.count() / total_rows * 100).round(2)
})

# מיון לפי אחוז ערכים חסרים (מהגבוה לנמוך)
missing_data = missing_data.sort_values('אחוז_חסר', ascending=False)

print("📋 פירוט ערכים חסרים לפי עמודה:")
print("-" * 60)
for idx, row in missing_data.iterrows():
    if row['ערכים_חסרים'] > 0:
        print(f"• {row['עמודה']}: {row['ערכים_חסרים']:,} חסרים ({row['אחוז_חסר']:.1f}%)")
    else:
        print(f"✅ {row['עמודה']}: שלמה לחלוטין")

# 3. סיכום סטטיסטי
print("\n" + "=" * 60)
print("📈 סיכום סטטיסטי:")

completely_missing = missing_data[missing_data['אחוז_חסר'] == 100]
very_high_missing = missing_data[(missing_data['אחוז_חסר'] >= 70) & (missing_data['אחוז_חסר'] < 100)]
high_missing = missing_data[(missing_data['אחוז_חסר'] >= 30) & (missing_data['אחוז_חסר'] < 70)]
moderate_missing = missing_data[(missing_data['אחוז_חסר'] >= 10) & (missing_data['אחוז_חסר'] < 30)]
low_missing = missing_data[(missing_data['אחוז_חסר'] > 0) & (missing_data['אחוז_חסר'] < 10)]
complete_columns = missing_data[missing_data['אחוז_חסר'] == 0]

print(f"🔴 עמודות ריקות לחלוטין (100%): {len(completely_missing)}")
if len(completely_missing) > 0:
    print(f"   → {', '.join(completely_missing['עמודה'].tolist())}")

print(f"🟠 עמודות עם ערכים חסרים גבוהים מאוד (70-99%): {len(very_high_missing)}")
if len(very_high_missing) > 0:
    print(f"   → {', '.join(very_high_missing['עמודה'].tolist())}")

print(f"🟡 עמודות עם ערכים חסרים גבוהים (30-69%): {len(high_missing)}")
if len(high_missing) > 0:
    print(f"   → {', '.join(high_missing['עמודה'].tolist())}")

print(f"🟤 עמודות עם ערכים חסרים בינוניים (10-29%): {len(moderate_missing)}")
if len(moderate_missing) > 0:
    print(f"   → {', '.join(moderate_missing['עמודה'].tolist())}")

print(f"🟢 עמודות עם ערכים חסרים נמוכים (1-9%): {len(low_missing)}")
if len(low_missing) > 0:
    print(f"   → {', '.join(low_missing['עמודה'].tolist())}")

print(f"✅ עמודות שלמות (0%): {len(complete_columns)}")
if len(complete_columns) > 0:
    print(f"   → {', '.join(complete_columns['עמודה'].tolist())}")



# 5. הצגת הטבלה המלאה כדאטה פריים
print(f"\n📋 טבלת ערכים חסרים מלאה:")
print("-" * 60)

# הצגת הדאטה פריים באופן אינטראקטיבי
display(missing_data)

print(f"\n" + "=" * 60)
print("✅ ניתוח ערכים חסרים הושלם")

🔍 ניתוח ערכים חסרים בנתוני הסקר
📊 מימדי הדאטה: 1,012 שורות, 17 עמודות
------------------------------------------------------------
📋 פירוט ערכים חסרים לפי עמודה:
------------------------------------------------------------
• הכנסה מעל או מתחת לממוצע: 847 חסרים (83.7%)
• ספק טלויזיה בינלאומי: 212 חסרים (20.9%)
• הגדרה דתית: 161 חסרים (15.9%)
✅ ת״ז: שלמה לחלוטין
✅ ספק טלויזיה ישראלי: שלמה לחלוטין
✅ איזור מגורים: שלמה לחלוטין
✅ הכנסה: שלמה לחלוטין
✅ מקור: שלמה לחלוטין
✅ סוג מכשיר: שלמה לחלוטין
✅ הכנסה למשק בית: שלמה לחלוטין
✅ מגדר: שלמה לחלוטין
✅ השכלה: שלמה לחלוטין
✅ ספק האינטרנט: שלמה לחלוטין
✅ סוג חיבור האינטרנט: שלמה לחלוטין
✅ לאום: שלמה לחלוטין
✅ גיל: שלמה לחלוטין
✅ צריכת שירותי מוזיקה: שלמה לחלוטין

📈 סיכום סטטיסטי:
🔴 עמודות ריקות לחלוטין (100%): 0
🟠 עמודות עם ערכים חסרים גבוהים מאוד (70-99%): 1
   → הכנסה מעל או מתחת לממוצע
🟡 עמודות עם ערכים חסרים גבוהים (30-69%): 0
🟤 עמודות עם ערכים חסרים בינוניים (10-29%): 2
   → ספק טלויזיה בינלאומי, הגדרה דתית
🟢 עמודות עם ערכים חסרים נמוכים (1-

,עמודה,ערכים_חסרים,אחוז_חסר,ערכים_קיימים,אחוז_קיים
הכנסה מעל או מתחת לממוצע,הכנסה מעל או מתחת לממוצע,847,83.70,165,16.30
ספק טלויזיה בינלאומי,ספק טלויזיה בינלאומי,212,20.95,800,79.05
הגדרה דתית,הגדרה דתית,161,15.91,851,84.09
ת״ז,ת״ז,0,0.00,1012,100.00
ספק טלויזיה ישראלי,ספק טלויזיה ישראלי,0,0.00,1012,100.00
איזור מגורים,איזור מגורים,0,0.00,1012,100.00
הכנסה,הכנסה,0,0.00,1012,100.00
מקור,מקור,0,0.00,1012,100.00
סוג מכשיר,סוג מכשיר,0,0.00,1012,100.00
הכנסה למשק בית,הכנסה למשק בית,0,0.00,1012,100.00



✅ ניתוח ערכים חסרים הושלם


# מילוי ערכים חסרים באסטרטגיות חכמות
לאחר שזיהינו את הערכים החסרים בנתונים, הגיע הזמן לטפל בהם באופן חכם. במקום למחוק שורות או למלא ערכים אקראיים, נשתמש באסטרטגיות מבוססות לוגיקה עסקית וטכניקות למידת מכונה.
עבור עמודת "הכנסה מעל או מתחת לממוצע" נוכל להשתמש בנתוני ההכנסה הקיימים ולהשוות אותם לשכר הממוצע במשק. 
עבור עמודות "ספק טלוויזיה בינלאומי" ו"הגדרה דתית" נשתמש באלגוריתם "שכן קרוב" שימצא דפוסים בין המשיבים הדומים ויחזה ערכים מתאימים.


In [54]:
# תיקון טיפול בעמודת הכנסה ומיזוג עמודות כפולות

import pandas as pd
import numpy as np

print("🔧 תיקון בעיות זוהו בנתונים")
print("=" * 60)

# 1. טיפול בעמודת הכנסה - המרת טווחים למספרים
print("💰 מעבד עמודת הכנסה עם טווחי ערכים...")

def income_range_to_number(income_str):
    """המרת טווח הכנסה למספר ממוצע"""
    if pd.isna(income_str) or income_str == 'לא יודע|ת':
        return None
    
    income_str = str(income_str).strip()
    
    # מיפוי טווחי הכנסה למספרים (ממוצע הטווח)
    income_mapping = {
        'עד 3000': 1500,
        '3001-4000': 3500,
        '4001-5000': 4500,
        '5001-6000': 5500,
        '6001-7000': 6500,
        '7001-8000': 7500,
        '8001-9000': 8500,
        '9001-10000': 9500,
        '10001-11000': 10500,
        '11001-12000': 11500,
        '12001-13000': 12500,
        '13001-14000': 13500,
        '14001-16000': 15000,
        '16001-18000': 17000,
        '18001-22000': 20000,
        '22001-25000': 23500,
        '25001-30000': 27500,
        '30001-35000': 32500,
        '35001-40000': 37500,
        '40001 ומעלה': 45000
    }
    
    return income_mapping.get(income_str, None)

# המרת עמודת ההכנסה למספרים
df['הכנסה_מספרית'] = df['הכנסה'].apply(income_range_to_number)

print(f"✅ הומרו טווחי הכנסה למספרים")
print(f"   ערכים מספריים זמינים: {df['הכנסה_מספרית'].notna().sum()}")

# 2. מילוי עמודת "הכנסה מעל או מתחת לממוצע" עם הנתונים המספריים
AVERAGE_SALARY = 12700

def classify_income_from_number(income_num, current_classification):
    """סיווג הכנסה מספרית לעומת הממוצע"""
    # אם יש כבר ערך, אל תשנה
    if pd.notna(current_classification):
        return current_classification
    
    if pd.isna(income_num):
        return None
    
    if income_num > AVERAGE_SALARY:
        return "מעט מעל הערך הממוצע"
    else:
        return "מעט מתחת לערך הממוצע"

# יישום הסיווג החדש
before_fill = df['הכנסה מעל או מתחת לממוצע'].isna().sum()

df['הכנסה מעל או מתחת לממוצע'] = df.apply(
    lambda row: classify_income_from_number(row['הכנסה_מספרית'], row['הכנסה מעל או מתחת לממוצע']), 
    axis=1
)

after_fill = df['הכנסה מעל או מתחת לממוצע'].isna().sum()
filled_income = before_fill - after_fill

print(f"💰 מולאו {filled_income} ערכים נוספים בעמודת הכנסה מעל/מתחת לממוצע")
print(f"   נותרו {after_fill} ערכים חסרים")

# 3. טיפול בעמודות הכפולות של ספק טלוויזיה בינלאומי
print(f"\n📺 מטפל בעמודות כפולות של ספק טלוויזיה בינלאומי...")

# בדיקה אילו עמודות קיימות
tv_col1 = 'ספק טלוויזיה בינלאומי'
tv_col2 = 'ספק טלויזיה בינלאומי'

if tv_col1 in df.columns and tv_col2 in df.columns:
    print(f"   נמצאו שתי עמודות: '{tv_col1}' ו-'{tv_col2}'")
    
    # איחוד שתי העמודות
    def merge_tv_columns(row):
        val1 = row[tv_col1] if pd.notna(row[tv_col1]) else None
        val2 = row[tv_col2] if pd.notna(row[tv_col2]) else None
        
        if val1 and val2:
            # אם יש ערך בשתי העמודות, נבחר את הראשונה
            return val1
        elif val1:
            return val1
        elif val2:
            return val2
        else:
            return None
    
    # יצירת עמודה מאוחדת
    df['ספק_טלוויזיה_בינלאומי_מאוחד'] = df.apply(merge_tv_columns, axis=1)
    
    # מחיקת העמודות הישנות
    df = df.drop(columns=[tv_col1, tv_col2])
    
    # שינוי שם העמודה החדשה
    df = df.rename(columns={'ספק_טלוויזיה_בינלאומי_מאוחד': 'ספק טלוויזיה בינלאומי'})
    
    print(f"✅ אוחדו שתי עמודות ספק טלוויזיה בינלאומי")
    print(f"   ערכים לא ריקים בעמודה המאוחדת: {df['ספק טלוויזיה בינלאומי'].notna().sum()}")

# 4. ניקוי עמודת ההכנסה המספרית (אופציונלי)
print(f"\n🧹 ניקוי נתונים...")
# אפשר למחוק את העמודה הזמנית אם לא נצרך אותה
# df = df.drop(columns=['הכנסה_מספרית'])

# 5. סיכום סופי
print(f"\n" + "=" * 60)
print("📊 סיכום תיקון הבעיות:")

final_missing = df.isnull().sum().sum()
print(f"   סה״כ ערכים חסרים לאחר התיקונים: {final_missing:,}")

remaining_missing = df.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0].sort_values(ascending=False)

if len(remaining_missing) > 0:
    print(f"\n   עמודות עם ערכים חסרים שנותרו:")
    for col, missing_count in remaining_missing.items():
        missing_pct = (missing_count / len(df)) * 100
        print(f"   • {col}: {missing_count:,} ({missing_pct:.1f}%)")

# הצגת התפלגות חדשה
print(f"\n📋 התפלגות מעודכנת - הכנסה מעל/מתחת לממוצע:")
income_dist = df['הכנסה מעל או מתחת לממוצע'].value_counts()
print(income_dist)

print(f"\n📋 התפלגות מעודכנת - ספק טלוויזיה בינלאומי:")
tv_dist = df['ספק טלוויזיה בינלאומי'].value_counts()
print(tv_dist.head(10))

print(f"\n✅ תיקון הבעיות הושלם בהצלחה")

🔧 תיקון בעיות זוהו בנתונים
💰 מעבד עמודת הכנסה עם טווחי ערכים...
✅ הומרו טווחי הכנסה למספרים
   ערכים מספריים זמינים: 749
💰 מולאו 689 ערכים נוספים בעמודת הכנסה מעל/מתחת לממוצע
   נותרו 158 ערכים חסרים

📺 מטפל בעמודות כפולות של ספק טלוויזיה בינלאומי...
   נמצאו שתי עמודות: 'ספק טלוויזיה בינלאומי' ו-'ספק טלויזיה בינלאומי'
✅ אוחדו שתי עמודות ספק טלוויזיה בינלאומי
   ערכים לא ריקים בעמודה המאוחדת: 1012

🧹 ניקוי נתונים...

📊 סיכום תיקון הבעיות:
   סה״כ ערכים חסרים לאחר התיקונים: 421

   עמודות עם ערכים חסרים שנותרו:
   • הכנסה_מספרית: 263 (26.0%)
   • הכנסה מעל או מתחת לממוצע: 158 (15.6%)

📋 התפלגות מעודכנת - הכנסה מעל/מתחת לממוצע:
מעט מתחת לערך הממוצע    387
מעט מעל הערך הממוצע     302
מסרב/ת להשיב            102
דומה לממוצע              27
קצת מעל הממוצע           14
הרבה מתחת לממוצע         11
קצת מתחת לממוצע           8
הרבה מעל הממוצע           3
Name: הכנסה מעל או מתחת לממוצע, dtype: int64

📋 התפלגות מעודכנת - ספק טלוויזיה בינלאומי:
נטפליקס – Netflix                                    

In [74]:
df.to_csv('C:\\Users\\USER001\\Desktop\\מטלות מבוא לתכנות רועי\\ISOCֹ-ILמטלת בית\\נתוני_סקר_מעובדים.csv', index=False, encoding='utf-8-sig')